In [1]:
!pip install transformers[torch]
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00


In [2]:
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import (AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoConfig)
import evaluate
import torch.nn as nn
import torch
import numpy as np
from utils import PROD_TOK, AUX_TOK

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and save data

## Split data

In [ ]:
#Load data
df = pd.read_hdf('human_data.h5', key='df')
df.head()

,Input_seq,Output,Type
0,"[5, 7, 4, 4, 16, 5, 5, 6, 5, 2, 17, 0, 13, 6, ...",[5],Piezas
1,"[1, 6, 8, 4, 1, 9, 11, 7, 2, 5, 8, 10, 1, 12, ...",[5],Productos
2,"[2, 1]",[3],Productos
3,"[9, 5, 6, 9, 11, 1, 2, 6, 8, 12, 0, 6, 7, 10, ...",[7],Productos


In [ ]:
#We divide the dataset into products and pieces
df_prod = df.loc[df.Type=='Productos']
df_piez = df.loc[df.Type=='Piezas']

In [ ]:
#Save splitted datasets for later use
df_prod.to_hdf('products_human.h5', key='df_prod', index=False)
df_piez.to_hdf('pieces_human.h5', key='df_piez', index=False)

<ipython-input-7-aa00f0db12e0>:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Input_seq', 'Output', 'Type'], dtype='object')]

  df_prod.to_hdf('products_human.h5', key='df_prod', index=False)
<ipython-input-7-aa00f0db12e0>:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Input_seq', 'Output', 'Type'], dtype='object')]

  df_piez.to_hdf('pieces_human.h5', key='df_piez', index=False)


## Process SQL data to suit format

In [ ]:
#Load SQL dataset
df_hist_piez = pd.read_csv('rl_piezas.csv')

In [ ]:
#Turn list of strings into array (formatting)
X_series = df_hist_piez['SIMUL'].apply(lambda x: [int(x.split(', ')[0][1:])] + [int(x.split(', ')[i+1]) for i in range(len(x.split(', '))-2)] + [int(x.split(', ')[-1][:-1])])
y_series = df_hist_piez['DES'].apply(lambda x: [int(x[1:-1])])
df_hist_piez.drop_duplicates('SIMUL', inplace=True, ignore_index=True)
df_hist_piez.drop(['SIMUL', 'DES'], axis=1,inplace=True)
df_hist_piez.insert(0, 'X', X_series)
df_hist_piez.insert(1, 'y', y_series)
df_hist_piez.head()

,X,y
0,"[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10...",[5]
1,"[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10...",[8]
2,"[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10...",[8]
3,"[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10...",[8]
4,"[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10...",[8]


In [ ]:
#Saved processed data
df_hist_piez.to_hdf('pieces_hist.h5', key='df_hist_piez')

<ipython-input-10-6992fe2cbc23>:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['X', 'y'], dtype='object')]

  df_hist_piez.to_hdf('pieces_hist.h5', key='df_hist_piez')


## Load data

In [ ]:
#Load splitted datasets
df_prod = pd.read_hdf('products_human.h5', key='df_prod')
df_piez = pd.read_hdf('pieces_human.h5', key='df_piez')

In [4]:
#Load SQL processed dataset
df_hist_piez = pd.read_hdf('pieces_hist.h5', key='df_hist_piez')

# Load and build model

In [5]:
#We define a custom model as our reward model
class RewardModel (torch.nn.Module):
    def __init__(self, model_name:str, hidden_size:int, classes:int,*model_args, **kwargs):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.vocab_size = classes+1
        self.config.pad_token_id = classes
        self.config.hidden_size = hidden_size
        #print(self.config)
        self.base = AutoModelForSequenceClassification.from_config(self.config)

        #print(self.base.classifier)
        #Last layer must be changed for it to be a regression problem
        self.base.classifier =  torch.nn.Linear(in_features=hidden_size, out_features=1, bias=True)

        #Only keep this line if the model was not pre-trained (it's a fresh BERT model)
        #PAD = max(list(AUX_TOK.values()))+1 #Change this line according to the model (pieces/products)
        #self.base.bert.embeddings.word_embeddings = torch.nn.Embedding(PAD+1, hidden_size, PAD)
        #self = self.base

    #Our custom model should take as input a pair of right/wrong answers with a fixed sequence
    def forward(self, input_ids, attention_mask=None,
                token_type_ids=None, train:bool=False):
        #Whether the model is being trained or tested is important when inferencing
        if train:
            #The input_ids are structured in a way in which the first chunk corresponds to right examples and the second one to wrong ones
            input_right, input_wrong  = input_ids[0][0], input_ids[0][1]
            out_right = self.base(input_ids=input_right, attention_mask=attention_mask, token_type_ids=token_type_ids)
            out_wrong = self.base(input_ids=input_wrong, attention_mask=attention_mask, token_type_ids=token_type_ids)
            return [out_right, out_wrong]
        else:
            return self.base(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    #Setup a custom .train() function because the base model will not train
    def train(self, mode:bool):
        self.base.train(mode=mode)
        return self.base

In [6]:
#We initialize our custom reward model
rm = RewardModel("VCNC/bert_piezas3", hidden_size=768, classes=20).to(device)

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

In [7]:
rm.base

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21, 768, padding_idx=20)
      (position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [8]:
#This is an example of how to inference the model with batches of data
input = torch.tensor([[[[2, 2, 1, 3, 4, 1], [3, 2, 1, 3, 4, 4]], [[2, 2, 1, 3, 4, 3], [3, 2, 1, 3, 4, 1]]]]).to(device)
token_type = torch.tensor([[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]]).to(device)
rm(input, token_type_ids=token_type, train=True)

[SequenceClassifierOutput(loss=None, logits=tensor([[0.1500],
         [0.0259]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None),
 SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0100],
         [-0.0986]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]

In [9]:
#We can see that the results do not vary, indicating that our functions are correct
input = torch.tensor([[2, 2, 1, 3, 4, 1], [3, 2, 1, 3, 4, 4]]).to(device)
token_type = torch.tensor([[0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1]]).to(device)
rm(input, token_type_ids=token_type, train=False)

SequenceClassifierOutput(loss=None, logits=tensor([[0.2177],
        [0.0751]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Preprocess data

In [8]:
#In this case we will use the product dataset
data = df_hist_piez.to_numpy()
keys = np.unique(np.array(list(AUX_TOK.values()))) #Get unique tokens only
dataset = np.zeros((len(data)*(len(keys)-1), 3), dtype=object) #The number of pairs is the number of examples at the beginning times 1 minus keys

#Iterate through the dataset to generate sets (correct, wrong, input)
for i in range(len(data)):
    internal_cont = 0
    for j in range(len(keys)):
        if j != data[i, 1][0]:
            dataset[i*(len(keys)-1)+internal_cont, 0] = data[i, 1][0]
            dataset[i*(len(keys)-1)+internal_cont, 1] = j
            dataset[i*(len(keys)-1)+internal_cont, 2] = data[i, 0]
            internal_cont += 1
dataset

array([[5, 0,
        list([5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10, 10, 10, 13, 8, 2, 2, 13, 0, 10, 5, 5, 8, 2, 2, 10, 7, 7, 0, 1, 0, 18, 10, 12, 10, 10, 10, 13, 13, 13, 13, 1, 5, 5, 17, 7, 8, 8, 5])],
       [5, 1,
        list([5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10, 10, 10, 13, 8, 2, 2, 13, 0, 10, 5, 5, 8, 2, 2, 10, 7, 7, 0, 1, 0, 18, 10, 12, 10, 10, 10, 13, 13, 13, 13, 1, 5, 5, 17, 7, 8, 8, 5])],
       [5, 2,
        list([5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 10, 10, 10, 13, 8, 2, 2, 13, 0, 10, 5, 5, 8, 2, 2, 10, 7, 7, 0, 1, 0, 18, 10, 12, 10, 10, 10, 13, 13, 13, 13, 1, 5, 5, 17, 7, 8, 8, 5])],
       ...,
       [10, 16,
        list([10, 10, 10, 13, 13, 8, 2, 2, 0, 0, 10, 10, 5, 10, 5, 8, 8, 2, 2, 10, 2, 2, 17, 5, 5, 5, 5, 8, 8, 8, 8, 8, 0, 5, 0, 0, 0, 19, 17, 17, 0, 15, 0, 10, 10, 10, 10, 7, 7, 19, 12, 12, 12, 12, 2, 13, 10, 8, 5, 5, 13, 13, 13, 13, 5, 4, 4, 8, 8, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 12, 5, 10, 10, 10, 4, 4, 5, 11, 5, 8, 8, 0, 0, 2, 1, 13, 13, 0, 0

## Data formatting and train/test splitting

In [9]:
MAX_LEN = max([len(i) for i in dataset[:, 2]])
PAD = max(list(AUX_TOK.values()))+1

input_ids = []
attention_mask = []
token_type_ids = []
train = []
for i in range(len(dataset)):
    pad_len = MAX_LEN-len(dataset[:, 2][i])
    right_seq = dataset[:, 2][i]+[dataset[i, 0]]+[PAD]*pad_len
    wrong_seq = dataset[:, 2][i]+[dataset[i, 1]]+[PAD]*pad_len
    input_ids.append([right_seq, wrong_seq]) #The input_ids is a concatenation of the right and wrong example
    attention_mask.append([1]*(len(dataset[:, 2][i])+1)+[0]*pad_len)
    token_type_ids.append([0]*len(dataset[:, 2][i])+[1]*(pad_len+1)) #The +1 is because the output selected by the human changes the token type
    train.append(True) #In RLHF every example is given as a training example REVISAR!!!
input_ids[0], attention_mask[0], token_type_ids[0], train[0] #Get first element

([[5,
   8,
   8,
   8,
   8,
   8,
   8,
   5,
   8,
   2,
   10,
   8,
   8,
   8,
   10,
   10,
   10,
   13,
   8,
   2,
   2,
   13,
   0,
   10,
   5,
   5,
   8,
   2,
   2,
   10,
   7,
   7,
   0,
   1,
   0,
   18,
   10,
   12,
   10,
   10,
   10,
   13,
   13,
   13,
   13,
   1,
   5,
   5,
   17,
   7,
   8,
   8,
   5,
   5,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20],
  [5,
   8,
   8,
   8,
   8,
   8,
   8,
   5,
   8,
   2,
   10,
   8,
   8,
   8,
   10,
   10,
   10,
   13,
   8,
   2,
   2,
   13,
   0,
   10,
   5,
   5,
   8,
   2,
   2,
   10,
   7,
   7,
   0,
   1,
   0,
   18,
   10,
   12,
   10,
   10,
   10,
   13,
   13,
   1

In [10]:
#We create a df to store the data properly
data = pd.DataFrame(data={'input_ids':input_ids, 'attention_mask':attention_mask, 'token_type_ids':token_type_ids, 'train':train})
data.head()

,input_ids,attention_mask,token_type_ids,train
0,"[[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
1,"[[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
2,"[[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
3,"[[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True
4,"[[5, 8, 8, 8, 8, 8, 8, 5, 8, 2, 10, 8, 8, 8, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",True


## DatasetDict creation

In [11]:
#We turn our datasets into DatasetDicts
train = Dataset.from_pandas(data.loc[data.train == True].reset_index(drop=True))
test = Dataset.from_pandas(data.loc[data.train == False].reset_index(drop=True))

ds = DatasetDict()

ds['train'] = train
ds['validation'] = test

ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'token_type_ids', 'train'],
        num_rows: 1764
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'token_type_ids', 'train'],
        num_rows: 0
    })
})

# Model training

In [12]:
#We redefine a custom loss that maximizes the difference between the rewards of positive and negative examples
def custom_loss(out_right:torch.tensor, out_wrong:torch.tensor, return_outputs:bool=False):
    #The idea behind the equations is better explained in the following article: https://medium.com/towards-generative-ai/reward-model-training-2209d1befb5f
    diff = out_right-out_wrong
    loss = torch.sum(-torch.log(torch.sigmoid(out_right-out_wrong)))
    return (loss, diff) if return_outputs else loss

In [13]:
#Finally, we train the model
rm.train(mode=True)
"Training!"

'Training!'

In [14]:
#We use Adam optimizer
optimizer = torch.optim.Adam(params=rm.parameters(), lr=3e-07)

In [20]:
EPOCHS = 20
batch_size = 60
batches_per_epoch = len(train) // batch_size
last_batch = len(train)%batch_size


for epoch in range(EPOCHS):
    for i in range(batches_per_epoch):
        start = i * batch_size

        # Take a batch
        input_batch = train[start:start+batch_size]
        inputs = np.array(input_batch['input_ids'])

        # Format input_ids to match model requirements
        input_ids = torch.tensor(np.array([[inputs[:, 0], inputs[:, 1]]])).to(device)
        attention_mask = torch.tensor(input_batch['attention_mask']).to(device)
        token_type_ids = torch.tensor(input_batch['token_type_ids']).to(device)

        # Forward pass through the model
        out = rm(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids, train = True)
        loss = custom_loss(out[0].logits, out[1].logits)

        if i%(batches_per_epoch//2) == 0:
            #print(f"Correct reward: {out[0].logits} Wrong reward: {out[1].logits}")
            print(f"Loss epoch {epoch} batch {i}: {loss}")

        # Backpropagate
        loss.backward()

        # Update weights
        optimizer.step()

    #Take the last training examples that fall out of the batches
    # Take a batch
    input_batch = train[-last_batch:]
    inputs = np.array(input_batch['input_ids'])

    # Format input_ids to match model requirements
    input_ids = torch.tensor(np.array([[inputs[:, 0], inputs[:, 1]]])).to(device)
    attention_mask = torch.tensor(input_batch['attention_mask']).to(device)
    token_type_ids = torch.tensor(input_batch['token_type_ids']).to(device)

    # Forward pass through the model
    out = rm(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids, train = True)
    loss = custom_loss(out[0].logits, out[1].logits)

    # Backpropagate
    loss.backward()

    # Update weights
    optimizer.step()

Loss epoch 0 batch 0: 42.014251708984375
Loss epoch 0 batch 14: 40.94215393066406
Loss epoch 0 batch 28: 41.53577423095703
Loss epoch 1 batch 0: 42.02200698852539
Loss epoch 1 batch 14: 42.086082458496094
Loss epoch 1 batch 28: 41.54685974121094
Loss epoch 2 batch 0: 42.053985595703125
Loss epoch 2 batch 14: 42.65813446044922
Loss epoch 2 batch 28: 40.84783172607422
Loss epoch 3 batch 0: 41.721107482910156
Loss epoch 3 batch 14: 41.7093391418457
Loss epoch 3 batch 28: 42.90867233276367
Loss epoch 4 batch 0: 42.08832550048828
Loss epoch 4 batch 14: 41.446022033691406
Loss epoch 4 batch 28: 40.052433013916016
Loss epoch 5 batch 0: 42.085594177246094
Loss epoch 5 batch 14: 42.98945236206055
Loss epoch 5 batch 28: 41.53170394897461
Loss epoch 6 batch 0: 40.957157135009766
Loss epoch 6 batch 14: 42.092655181884766
Loss epoch 6 batch 28: 41.379276275634766
Loss epoch 7 batch 0: 42.347679138183594
Loss epoch 7 batch 14: 41.63581848144531
Loss epoch 7 batch 28: 42.07697296142578
Loss epoch 8 b

In [ ]:
loss

tensor(42.0421, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
rm.eval()

In [ ]:
#Evaluate if training was done correctly taking some training examples
inp = train[0:3]
inputs = np.array(inp['input_ids'])
input_ids = torch.tensor(np.array([[inputs[:, 0], inputs[:, 1]]])).to(device)
attention_mask = torch.tensor(inp['attention_mask']).to(device)
token_type_ids = torch.tensor(inp['token_type_ids']).to(device)
rm(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids, train = True)

[SequenceClassifierOutput(loss=None, logits=tensor([[-0.0626],
         [-0.3864],
         [-0.2346]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None),
 SequenceClassifierOutput(loss=None, logits=tensor([[-0.1413],
         [-0.2106],
         [-0.2052]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]

# Save and load model

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
#Save model
torch.save(rm.state_dict(), 'drive/MyDrive/Colab Notebooks/Auto-CNC/rm_dict.pt')

In [ ]:
#Load saved model
load_model = RewardModel("VCNC/bert_piezas3", hidden_size=768, classes=20).to(device)
load_model.load_state_dict(torch.load('rm/rm_dict.pt'))

<All keys matched successfully>

In [ ]:
load_model.eval()

In [ ]:
#Evaluate if model was loaded succesfully
inp = train[0:3]
inputs = np.array(inp['input_ids'])
input_ids = torch.tensor(np.array([[inputs[:, 0], inputs[:, 1]]])).to(device)
attention_mask = torch.tensor(inp['attention_mask']).to(device)
token_type_ids = torch.tensor(inp['token_type_ids']).to(device)
load_model(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids, train = True)

[SequenceClassifierOutput(loss=None, logits=tensor([[0.7354],
         [0.7354],
         [0.7354]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None),
 SequenceClassifierOutput(loss=None, logits=tensor([[0.7354],
         [0.7354],
         [0.7354]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)]